In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Ideas:
- ampliar rango dinamico 
- Podriamos hacer homogenizar el histograma
- Filtro de suavizado 
- Filtro de sal y pimienta(sobre todo sal y pimienta)

como subir documentos
git pull
modificar
git add . ( o nombre del archivo)
git commit -m "descripcion de los cambios realizados"